In [1]:
import pandas as pd
import torch
import torch.nn as nn

In [2]:
dataset = pd.read_csv("dataset/SampleData.csv")
dataset.head()

<IPython.core.display.Javascript object>

,Ingredient,Product,QMerged_label,Real_Cost,V_0
0,1,1,11,10,0
1,1,2,12,6,0
2,2,1,21,8,0
3,2,2,22,11,0
4,3,1,31,3,0


In [10]:
class MonteCarlo:
    def __init__(self, dataset, num_episodes, epsilon, budget, reward, alpha):
        self.state_space = list(set(dataset['Ingredient']))
        

In [11]:
driver = MonteCarlo(dataset=dataset, num_episodes=10, epsilon=0.9, budget=100, reward=1, alpha=0.5)

[1, 2, 3, 4]


In [ ]:
class DQNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNetwork, self).__init__()
        self.layer1 = nn.Linear(state_size, 64)
        self.layer2 = nn.Linear(64,action_size)


    def feed_forward(self, state):
        x = torch.relu(self.layer1(state))
        return self.layer2(x)
